## ANALYZE
the data

This is a work in progress  

Vectorize the data in beer.review  
Diminish the importance of common words  
Compare ML algorithms to use the review data to predict beer.style  

In [26]:
# IMPORT MODULES AND THE DATA SET
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split 
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

df = pd.read_csv('beer.csv', header=0)
df_copy = df  #save a copy of dataframe for reference. 
print('length',len(df))
pd.set_option('max_colwidth', 220)
df.head(3)

length 80818


,name,brewery,style,rating,review
0,Big Rock Ale,Big Rock Brewery,Scottish Ale,3.90,"smell  soft hop aroma with significant malt scents. this one smells very creamy. taste  and creamy it is. the traditional irish flavors come out at the tongue. this is creamy, not like a cream ale, but close. the m..."
1,Flip Ale,Dogfish Head Craft Brewery,Old Ale,4.08,on tap at dfh rehoboth... collab with eatily... cardamom and red wine must. golden orange. .no head. typical dfh yeast aroma. ..some spice and maybe a belgian influence. sweet spicy and somewhat fruity.. not much ol...
2,The Almond Marzen Project - Beer Camp #26,Sierra Nevada Brewing Co.,Märzen / Oktoberfest,3.78,"nice auburn impressions, tons of clarity, solid inch of off white head. aroma was a little bit sweet and nutty. taste gave a little more sweetness, stayed away from hops and bitterness, relatively light bodied. no..."


In [27]:
df['review'][7]

' '

In [57]:
# DATA PREP
print('df original length',len(df))
# drop all reviews with < 20 characters
df = df[df['review'].map(len) > 20]
print('length without short reviews',len(df))

# remove uncommon styles (in EDA, I found 13 styles with fewer than 60 reviews)
#uncommon = [ 'American Dark Wheat Ale','Bière de Champagne / Bière Brut', 'Black & Tan', 
#            'Eisbock', 'Faro', 'Gueuze', 'Happoshu', 'Japanese Rice Lager', 'Kristalweizen',
#            'Kvass', 'Lambic - Unblended','Roggenbier', 'Sahti' ]   
labels = df.groupby(['style']).size() 
uncommon = labels[labels<130]
df = df.loc[~df['style'].isin(uncommon.index)]
print('length without uncommon styles',len(df))

# reset dataframe index for the shortened dataframe
df['index'] = np.arange(len(df))
df = df.set_index('index')

# Change review to a string of words.  remove non-letters, make lower case, split into words.  
# Remove stopwords (common words.)  Join back together into a long string of words.  
def review_to_words(review):
    letters_only = re.sub('[^a-zA-Z]',' ', review)
    words = letters_only.lower().split()
    stops = set(stopwords.words('english'))  
    good_words = [w for w in words if not w in stops]
    return(' '.join(good_words))

# clean the reviews
df['clean_review'] = df['review'].apply(review_to_words)

df.head(3)

df original length 46861
length without short reviews 46861
length without uncommon styles 46861


,name,brewery,style,rating,review,clean_review,review_length,avg_word_length
index,,,,,,,,
0,Big Rock Ale,Big Rock Brewery,Scottish Ale,3.90,"smell  soft hop aroma with significant malt scents. this one smells very creamy. taste  and creamy it is. the traditional irish flavors come out at the tongue. this is creamy, not like a cream ale, but close. the m...",smell soft hop aroma significant malt scents one smells creamy taste creamy traditional irish flavors come tongue creamy like cream ale close malt big buttery smooth hops unique sharp hop flavor easy saturated well m...,720,5.885714
1,Flip Ale,Dogfish Head Craft Brewery,Old Ale,4.08,on tap at dfh rehoboth... collab with eatily... cardamom and red wine must. golden orange. .no head. typical dfh yeast aroma. ..some spice and maybe a belgian influence. sweet spicy and somewhat fruity.. not much ol...,tap dfh rehoboth collab eatily cardamom red wine must golden orange head typical dfh yeast aroma spice maybe belgian influence sweet spicy somewhat fruity much old ale characteristic light still tasty cardamom add ni...,380,5.512195
2,The Almond Marzen Project - Beer Camp #26,Sierra Nevada Brewing Co.,Märzen / Oktoberfest,3.78,"nice auburn impressions, tons of clarity, solid inch of off white head. aroma was a little bit sweet and nutty. taste gave a little more sweetness, stayed away from hops and bitterness, relatively light bodied. no...",nice auburn impressions tons clarity solid inch white head aroma little bit sweet nutty taste gave little sweetness stayed away hops bitterness relatively light bodied nothing almond came obvious kind fancied oktober...,5094,5.641434


In [58]:
df.name.size

46861

In [59]:
# ADDITIONAL FEATURE ENGINEERING
# review length
df['review_length'] = df['review'].apply(len)

# average word length
def avg_word_len(words):
    separate_words = words.split()
    count_words = (len(separate_words))    # number of words
    if count_words> 0:
        characters = len(words)  # length of text
        avg = (characters - count_words+1)/count_words
    else:
        avg = 5.65  # this is the mean of 49000 reviews    
    return avg   

df['avg_word_length'] = df['clean_review'].apply(avg_word_len)

In [60]:
print('mean:',df['review_length'].mean())
print('max: ',df['review_length'].max())
print('min: ',df['review_length'].min())
print('')
print('mean:',df['avg_word_length'].mean())
print('max: ',df['avg_word_length'].max())
print('min: ',df['avg_word_length'].min())

mean: 1909.6980004694735
max:  22085
min:  21

mean: 5.683280547075606
max:  11.0
min:  3.0


In [61]:
styles = df.groupby(['style']).size() 
print('Number of styles used:', len(styles))
print('')
print(styles.index)

Number of styles used: 57

Index(['Altbier', 'American Amber / Red Ale', 'American Black Ale',
       'American Blonde Ale', 'American Double / Imperial IPA',
       'American Lager', 'American Pale Wheat Ale', 'American Strong Ale',
       'American Wild Ale', 'Barleywine', 'Belgian Dark Ale',
       'Belgian Strong Dark Ale', 'Belgian Strong Pale Ale',
       'Berliner Weissbier', 'Bitter', 'Bock', 'Brown Ale', 'Cream Ale',
       'Czech Pilsener', 'Doppelbock', 'Dubbel', 'Dunkelweizen',
       'Euro Dark Lager', 'Euro Pale Lager', 'Farm Ale',
       'Fruit / Vegetable Beer', 'German Pilsener', 'Gose', 'Hefeweizen',
       'Herbed / Spiced Beer', 'IPA', 'Imperial Stout', 'Irish Dry Stout',
       'Irish Red Ale', 'Kellerbier / Zwickelbier', 'Kölsch', 'Lager',
       'Lambic - Fruit', 'Light Lager', 'Maibock / Helles Bock',
       'Munich Dunkel Lager', 'Munich Helles Lager', 'Märzen / Oktoberfest',
       'Old Ale', 'Pale Ale', 'Porter', 'Pumpkin Ale', 'Quadrupel (Quad)',
       'Rus

In [70]:
df['style'].replace('Saison / Farmhouse Ale', 'Farm Ale', inplace=True)
df['style'].replace('Bière de Garde', 'Farm Ale', inplace=True)
df['style'].replace('American IPA', 'IPA', inplace=True)
df['style'].replace('English India Pale Ale (IPA)', 'IPA', inplace=True)
df['style'].replace('Belgian IPA', 'IPA', inplace=True)
df['style'].replace('Scotch Ale / Wee Heavy', 'Scottish Ale', inplace=True)
df['style'].replace('American Pale Ale (APA)', 'Pale Ale', inplace=True)
df['style'].replace('English Pale Ale', 'Pale Ale', inplace=True)
df['style'].replace('Belgian Pale Ale', 'Pale Ale', inplace=True)
df['style'].replace('American Brown Ale', 'Brown Ale', inplace=True)
df['style'].replace('English Brown Ale', 'Brown Ale', inplace=True)
df['style'].replace('English Dark Mild Ale', 'Brown Ale', inplace=True)
df['style'].replace('American Stout', 'Stout', inplace=True)
df['style'].replace('English Stout', 'Stout', inplace=True)
df['style'].replace('Milk / Sweet Stout', 'Stout', inplace=True)
df['style'].replace('Oatmeal Stout', 'Stout', inplace=True)
df['style'].replace('Oatmeal Stout', 'Stout', inplace=True)
df['style'].replace('American Double / Imperial Stout', 'Imperial Stout', inplace=True)
df['style'].replace('Russian Imperial', 'Imperial Stout', inplace=True)
df['style'].replace('American Porter', 'Porter', inplace=True)
df['style'].replace('Baltic Porter', 'Porter', inplace=True)
df['style'].replace('English Porter', 'Porter', inplace=True)
df['style'].replace('American Amber / Red Lager', 'Lager', inplace=True)
df['style'].replace('Vienna Lager', 'Lager', inplace=True)
df['style'].replace('German Pilsener', 'Lager', inplace=True)
df['style'].replace('Munich Helles Lager', 'Lager', inplace=True)
df['style'].replace('American Adjunct Lager', 'American Lager', inplace=True)
df['style'].replace('American Pale Lager', 'American Lager', inplace=True)
df['style'].replace('American Barleywine', 'Barleywine', inplace=True)
df['style'].replace('English Barleywine', 'Barleywine', inplace=True)
df['style'].replace('English Bitter', 'Bitter', inplace=True)
df['style'].replace('Extra Special / Strong Bitter (ESB)', 'Bitter', inplace=True)
df['style'].replace('American Pale Wheat Ale', 'Wheat', inplace=True)
df['style'].replace('Witbier', 'Wheat', inplace=True)
df['style'].replace('Witbier', 'Wheat', inplace=True)

In [71]:
styles = df.groupby(['style']).size() 
print('Number of styles used:', len(styles))
print('')
print(styles.index)

Number of styles used: 54

Index(['Altbier', 'American Amber / Red Ale', 'American Black Ale',
       'American Blonde Ale', 'American Double / Imperial IPA',
       'American Lager', 'American Strong Ale', 'American Wild Ale',
       'Barleywine', 'Belgian Dark Ale', 'Belgian Strong Dark Ale',
       'Belgian Strong Pale Ale', 'Berliner Weissbier', 'Bitter', 'Bock',
       'Brown Ale', 'Cream Ale', 'Czech Pilsener', 'Doppelbock', 'Dubbel',
       'Dunkelweizen', 'Euro Dark Lager', 'Euro Pale Lager', 'Farm Ale',
       'Fruit / Vegetable Beer', 'Gose', 'Hefeweizen', 'Herbed / Spiced Beer',
       'IPA', 'Imperial Stout', 'Irish Dry Stout', 'Irish Red Ale',
       'Kellerbier / Zwickelbier', 'Kölsch', 'Lager', 'Lambic - Fruit',
       'Light Lager', 'Maibock / Helles Bock', 'Munich Dunkel Lager',
       'Märzen / Oktoberfest', 'Old Ale', 'Pale Ale', 'Porter', 'Pumpkin Ale',
       'Quadrupel (Quad)', 'Russian Imperial Stout', 'Rye Beer', 'Schwarzbier',
       'Scottish Ale', 'Smoked Bee

In [72]:
df.head(3)

,name,brewery,style,rating,review,clean_review,review_length,avg_word_length
index,,,,,,,,
0,Big Rock Ale,Big Rock Brewery,Scottish Ale,3.90,"smell  soft hop aroma with significant malt scents. this one smells very creamy. taste  and creamy it is. the traditional irish flavors come out at the tongue. this is creamy, not like a cream ale, but close. the m...",smell soft hop aroma significant malt scents one smells creamy taste creamy traditional irish flavors come tongue creamy like cream ale close malt big buttery smooth hops unique sharp hop flavor easy saturated well m...,720,5.885714
1,Flip Ale,Dogfish Head Craft Brewery,Old Ale,4.08,on tap at dfh rehoboth... collab with eatily... cardamom and red wine must. golden orange. .no head. typical dfh yeast aroma. ..some spice and maybe a belgian influence. sweet spicy and somewhat fruity.. not much ol...,tap dfh rehoboth collab eatily cardamom red wine must golden orange head typical dfh yeast aroma spice maybe belgian influence sweet spicy somewhat fruity much old ale characteristic light still tasty cardamom add ni...,380,5.512195
2,The Almond Marzen Project - Beer Camp #26,Sierra Nevada Brewing Co.,Märzen / Oktoberfest,3.78,"nice auburn impressions, tons of clarity, solid inch of off white head. aroma was a little bit sweet and nutty. taste gave a little more sweetness, stayed away from hops and bitterness, relatively light bodied. no...",nice auburn impressions tons clarity solid inch white head aroma little bit sweet nutty taste gave little sweetness stayed away hops bitterness relatively light bodied nothing almond came obvious kind fancied oktober...,5094,5.641434


In [73]:
avg_word_len(df['clean_review'][12])


5.841269841269841

In [74]:
df.shape

(46861, 8)

In [75]:
# max(len(w) for w in words) 
# or max(map(len, words))
# max word length
def find_longest_word(words):
#    separate_words = words.split()
    return(max([len(word) for word in words]) )
   
long = find_longest_word(df['review'][2])
print(long)

df['max_word'] = find_longest_word(df['clean_review'])

1


In [76]:
# split into train and test data
X = df['clean_review'].values
y = df['style'].values
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.3, random_state=22)
print('X_train:',len(X_train), 'y_train:',len(y_train))
print('X_test:', len(X_test), 'y_test:', len(y_test))
print('X_train shape:',X_train.shape)
print('y_train shape:',y_train.shape)

X_train: 32802 y_train: 32802
X_test: 14059 y_test: 14059
X_train shape: (32802,)
y_train shape: (32802,)


In [77]:
y_train[:20]

array(['Stout', 'American Amber / Red Ale', 'American Amber / Red Ale',
       'Russian Imperial Stout', 'American Lager', 'Pale Ale',
       'Barleywine', 'Pale Ale', 'American Double / Imperial IPA',
       'American Amber / Red Ale', 'Farm Ale', 'IPA', 'IPA',
       'American Double / Imperial IPA', 'Light Lager', 'Hefeweizen',
       'Porter', 'American Wild Ale', 'Pale Ale', 'American Blonde Ale'], dtype=object)

In [78]:
# CALCULATE PREDICTION FOR ONE STYLE
# from kaggle.com/adamschroeder/countvectorizer-tfidfvectorizer-predict-comments
# ValueError: Found input variables with inconsistent numbers of samples: [32802, 3857]
# when calling cross_val_score.  I reduced df length in "train_target = y_train[y_train==...]
from sklearn.feature_extraction.text import TfidfVectorizer
word_vect = TfidfVectorizer()
word_vect.fit(X_train)
X_train_word_features = word_vect.transform(X_train)
test_features = word_vect.transform(X_test)

style_names = ['American IPA','American Porter'] 
# ['American Pale Ale (APA)','Saison / Farmhouse Ale','American Double / Imperial IPA']
losses = []
auc = []
for style_name in style_names:
    #call the labels one at a time so we can run the classifier on them
    train_target = y_train[y_train==style_name]
    test_target = y_test[y_test==style_name]
    classifier = LogisticRegression(solver='sag', C=10)
    cv_loss = np.mean(cross_val_score(classifier, X_train_word_features, 
                                      train_target, cv=5, scoring='neg_log_loss'))
    losses.append(cv_loss)
    print('CV Log_loss score for class {} is {}'.format(style_name, cv_loss))
    cv_score = np.mean(cross_val_score(classifier, X_train_word_features, 
                                       train_target, cv=5, scoring='accuracy'))
    print('CV Accuracy score for class {} is {}'.format(style_name, cv_score))
    
    classifier.fit(X_train, train_target)
    y_pred = classifier.predict(test_features)
    y_pred_prob = classifier.predict_proba(test_features)[:, 1]
    auc_score = metrics.roc_auc_score(test_target, y_pred_prob)
    auc.append(auc_score)
    print("CV ROC_AUC score {}\n".format(auc_score))
    
    print(confusion_matrix(test_target, y_pred))
    print(classification_report(test_target, y_pred))

print('Total average CV Log_loss score is {}'.format(np.mean(losses)))
print('Total average CV ROC_AUC score is {}'.format(np.mean(auc)))    


NameError: name 'cross_val_score' is not defined

In [215]:
df['review'][0]

'smell \x96 soft hop aroma with significant malt scents. this one smells very creamy. taste \x96 and creamy it is. the traditional irish flavors come out at the tongue. this is creamy, not like a cream ale, but close. the malt is big, buttery, and very smooth. the hops are very unique. this is not a sharp hop flavor. this is an easy, saturated, well-mixed blend that plays a complimenting second fiddle to the malt base. no sweetness. the finish is nutty and big. this ale changes personalities at the end. mouthfeel \x96 lightly carbonated and exceptionally smooth and creamy. drinkability \x96 very creamy. i know that i\x92m repeating myself, but this ale is creamy like peanut butter. it\x92s actually kind of nutty to boot. creamy. '

In [216]:
df['clean_review'][0]

'smell soft hop aroma significant malt scents one smells creamy taste creamy traditional irish flavors come tongue creamy like cream ale close malt big buttery smooth hops unique sharp hop flavor easy saturated well mixed blend plays complimenting second fiddle malt base sweetness finish nutty big ale changes personalities end mouthfeel lightly carbonated exceptionally smooth creamy drinkability creamy know repeating ale creamy like peanut butter actually kind nutty boot creamy'

In [217]:
print(len(uncommon), 'styles excluded:')
print(uncommon.index)

30 styles excluded:
Index(['American Dark Wheat Ale', 'American Double / Imperial Pilsner',
       'American Malt Liquor', 'Bière de Champagne / Bière Brut',
       'Black & Tan', 'Braggot', 'California Common / Steam Beer',
       'Chile Beer', 'Dortmunder / Export Lager', 'Eisbock',
       'English Pale Mild Ale', 'English Strong Ale', 'Euro Strong Lager',
       'Faro', 'Flanders Oud Bruin', 'Flanders Red Ale',
       'Foreign / Export Stout', 'Gueuze', 'Happoshu', 'Japanese Rice Lager',
       'Kristalweizen', 'Kvass', 'Lambic - Unblended', 'Low Alcohol Beer',
       'Rauchbier', 'Roggenbier', 'Sahti',
       'Scottish Gruit / Ancient Herbed Ale', 'Weizenbock', 'Wheatwine'],
      dtype='object', name='style')


In [218]:
beer_styles = df.groupby(['style']).size() 
print(len(beer_styles), 'styles included:')
print(beer_styles.index)

74 styles included:
Index(['Altbier', 'American Adjunct Lager', 'American Amber / Red Ale',
       'American Amber / Red Lager', 'American Barleywine',
       'American Black Ale', 'American Blonde Ale', 'American Brown Ale',
       'American Double / Imperial IPA', 'American Double / Imperial Stout',
       'American IPA', 'American Pale Ale (APA)', 'American Pale Lager',
       'American Pale Wheat Ale', 'American Porter', 'American Stout',
       'American Strong Ale', 'American Wild Ale', 'Baltic Porter',
       'Belgian Dark Ale', 'Belgian IPA', 'Belgian Pale Ale',
       'Belgian Strong Dark Ale', 'Belgian Strong Pale Ale',
       'Berliner Weissbier', 'Bière de Garde', 'Bock', 'Cream Ale',
       'Czech Pilsener', 'Doppelbock', 'Dubbel', 'Dunkelweizen',
       'English Barleywine', 'English Bitter', 'English Brown Ale',
       'English Dark Mild Ale', 'English India Pale Ale (IPA)',
       'English Pale Ale', 'English Porter', 'English Stout',
       'Euro Dark Lager', 'Euro Pal

In [7]:
### NOT USED.  I MOVED IT TO DATA PREP, above
# Change review to a string of words.  remove non-letters, make lower case, split into words.  
# Remove stopwords (common words.)  Join back together into a long string of words.  

def review_to_words(review):
    letters_only = re.sub('[^a-zA-Z]',' ', review)
    words = letters_only.lower().split()
    stops = set(stopwords.words('english'))  
    good_words = [w for w in words if not w in stops]
    return(' '.join(good_words))

# clean up train reviews
num_reviews = len(X_train)
clean_train_reviews = []
for b in range(0, num_reviews):
    clean_train_reviews.append(review_to_words(X_train[b]))
    
#clean up test reviews
num_reviews = len(X_test)
clean_test_reviews = []
for b in range(0, num_reviews):
    clean_test_reviews.append(review_to_words(X_test[b]))

In [104]:
# vectorize the train data, fit and transform into feature vectors
vectorizer = CountVectorizer(analyzer='word', min_df=7)
train_data_features = vectorizer.fit_transform(X_train)
# arrays are easy to work with, so:
#train_data_features = train_data_features.toarray()

In [105]:
vocab = vectorizer.get_feature_names()
print('first 100 words:', vocab[:100])

first 100 words: ['aa', 'aal', 'aals', 'aaron', 'ab', 'aba', 'aback', 'abandon', 'abandoned', 'abarwithnoname', 'abate', 'abates', 'abbaye', 'abbey', 'abbot', 'abbreviated', 'abby', 'abc', 'abd', 'abe', 'abetted', 'abetting', 'abilities', 'ability', 'abit', 'abita', 'able', 'ably', 'abner', 'abnormal', 'abnormally', 'aboard', 'abomination', 'abound', 'abounding', 'abounds', 'abovementioned', 'abrasion', 'abrasive', 'abrasively', 'abrasiveness', 'abraxas', 'abroad', 'abrupt', 'abruptly', 'abs', 'absence', 'absent', 'absinthe', 'absolute', 'absolutely', 'absolutly', 'absorb', 'absorbed', 'absorbing', 'absorbs', 'abstract', 'absurd', 'absurdly', 'abt', 'abundance', 'abundant', 'abundantly', 'abuse', 'abused', 'abv', 'abvs', 'abw', 'abysmal', 'abyss', 'ac', 'acacia', 'academy', 'acbf', 'accent', 'accented', 'accenting', 'accents', 'accentuate', 'accentuated', 'accentuates', 'accentuating', 'accept', 'acceptable', 'accepted', 'access', 'accessibility', 'accessible', 'accident', 'accidental'

In [79]:
text_clf= Pipeline([('vect', CountVectorizer(min_df=7)),
                   ('tfidf', TfidfTransformer()),
                   ('clf',MultinomialNB()) ])
# first pass, .1195,  after word cleaning .2076,  after combining styles .2639
text_clf = text_clf.fit(X_train, y_train)
predicted = text_clf.predict(X_test)
np.mean(predicted == y_test)

0.26388790098869053

In [80]:
# RANDOM FOREST 
#first pass 0.45, after cleaning 0.426, after combined styles 0.462

X_train2 = X_train[:5000]
y_train2 = y_train[:5000]
X_test2 = X_test[10000:10500]
y_test2 = y_test[10000:10500]

from sklearn.ensemble import RandomForestClassifier as RFC
text_clf= Pipeline([('vect', CountVectorizer(min_df=5)),
                   ('tfidf', TfidfTransformer()),
                   ('forest',RFC(n_estimators=300)) ])
text_clf = text_clf.fit(X_train2, y_train2)
predicted = text_clf.predict(X_test2)
np.mean(predicted == y_test2)

0.46200000000000002

In [81]:
# LOGISTIC REGRESSION
#after cleaning, 0.575,  after combined styles 0.6218
from sklearn.model_selection import cross_val_score

# vectorize the train data, fit and transform into feature vectors
text_clf= Pipeline([('vect', CountVectorizer(min_df=5)),
                   ('tfidf', TfidfTransformer()),
                   ('clf',LogisticRegression()) ])
text_clf = text_clf.fit(X_train, y_train)
predicted = text_clf.predict(X_test)
np.mean(predicted == y_test)

#scores = cross_val_score(LogisticRegression(), train_data_features, y_train, cv=5)
#print('mean cross-val accuracy: {:.2f}'.format(np.mean(scores)))

0.62180809445906537

In [ ]:
###
#this is quite slow.  don't run it until I can improve speed (min_df=5 is a start)
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

param_grid = {'C': [0.001,0.01, 0.1, 1,10]}
grid = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid.fit(X_train, y_train)
print('best cross-val score: {:.2f}'.format(grid.gest_score_))
print('best params:', grid.best_params_)

In [ ]:
num_reviews = len(X_test)
clean_test_reviews = []
for b in range(0, num_reviews):
    clean_test_reviews.append(review_to_words(X_test[b]))
    
test_data_features = vectorizer.transform(X_test)
test_data_features = test_data_features.toarray()
result = forest.predict(test_data_features)
output = pd.DataFrame(data = {'id':test['id'], 'style':result})
output_to_csv('BagOfWords_model.csv', index=False, quoting=3)

In [118]:
print(len(str(vectorizer.vocabulary_)))
print(str(vectorizer.vocabulary_)[:600])

5426
{'nice': 220, 'auburn': 31, 'impressions': 163, 'tons': 350, 'of': 232, 'clarity': 72, 'solid': 308, 'inch': 165, 'off': 233, 'white': 372, 'head': 144, 'aroma': 28, 'was': 364, 'little': 192, 'bit': 45, 'sweet': 327, 'and': 21, 'nutty': 229, 'taste': 331, 'gave': 125, 'more': 213, 'sweetness': 328, 'stayed': 316, 'away': 33, 'from': 121, 'hops': 153, 'bitterness': 47, 'relatively': 278, 'light': 188, 'bodied': 50, 'nothing': 226, 'almond': 11, 'came': 61, 'out': 240, 'it': 171, 'that': 335, 'obvious': 230, 'kind': 177, 'fancied': 112, 'up': 358, 'oktoberfest': 234, 'while': 371, 'good': 132, 


In [119]:
X2 = [df['review'][0]] 

vectorizer = CountVectorizer()
vectorizer.fit(X)
vector = vectorizer.transform(X2)
# summarize encoded vector
print(vector.shape)
print(type(vector))
print(vector.toarray())

(1, 384)
<class 'scipy.sparse.csr.csr_matrix'>
[[0 0 0 0 1 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 1 5 0 0 0 0 0 1 1 0 2 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
  0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 7 2 0 0 0 0 0 1 0 1 0 0 0 0 0
  0 0 0 0 0 2 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1
  0 0 0 2 0 0 0 2 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 1 1 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0
  0 0 2 7 0 0 0 0 0 6 0 0 0 0 2 0 0 0 0 0 1 0 0 0 1 0 0 0 0 4 0 0 0 0 0 0 0
  0 0 0 0 1 0 0 0 0 0 0 0 0 0]]


In [120]:
print(len(str(vectorizer.vocabulary_)))
print(str(vectorizer.vocabulary_)[:600])

5426
{'nice': 220, 'auburn': 31, 'impressions': 163, 'tons': 350, 'of': 232, 'clarity': 72, 'solid': 308, 'inch': 165, 'off': 233, 'white': 372, 'head': 144, 'aroma': 28, 'was': 364, 'little': 192, 'bit': 45, 'sweet': 327, 'and': 21, 'nutty': 229, 'taste': 331, 'gave': 125, 'more': 213, 'sweetness': 328, 'stayed': 316, 'away': 33, 'from': 121, 'hops': 153, 'bitterness': 47, 'relatively': 278, 'light': 188, 'bodied': 50, 'nothing': 226, 'almond': 11, 'came': 61, 'out': 240, 'it': 171, 'that': 335, 'obvious': 230, 'kind': 177, 'fancied': 112, 'up': 358, 'oktoberfest': 234, 'while': 371, 'good': 132, 


In [121]:
vector = vectorizer.transform(X2)
print(vector.toarray())

[[0 0 0 0 1 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 1 5 0 0 0 0 0 1 1 0 2 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
  0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 7 2 0 0 0 0 0 1 0 1 0 0 0 0 0
  0 0 0 0 0 2 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1
  0 0 0 2 0 0 0 2 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 1 1 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0
  0 0 2 7 0 0 0 0 0 6 0 0 0 0 2 0 0 0 0 0 1 0 0 0 1 0 0 0 0 4 0 0 0 0 0 0 0
  0 0 0 0 1 0 0 0 0 0 0 0 0 0]]


In [122]:
from sklearn.feature_extraction.text import TfidfVectorizer
# create the transform
vectorizer = TfidfVectorizer()
# tokenize and build vocab
vectorizer.fit(X)
# summarize
print(len(str(vectorizer.vocabulary_)))
print(str(vectorizer.vocabulary_)[:600])

5426
{'nice': 220, 'auburn': 31, 'impressions': 163, 'tons': 350, 'of': 232, 'clarity': 72, 'solid': 308, 'inch': 165, 'off': 233, 'white': 372, 'head': 144, 'aroma': 28, 'was': 364, 'little': 192, 'bit': 45, 'sweet': 327, 'and': 21, 'nutty': 229, 'taste': 331, 'gave': 125, 'more': 213, 'sweetness': 328, 'stayed': 316, 'away': 33, 'from': 121, 'hops': 153, 'bitterness': 47, 'relatively': 278, 'light': 188, 'bodied': 50, 'nothing': 226, 'almond': 11, 'came': 61, 'out': 240, 'it': 171, 'that': 335, 'obvious': 230, 'kind': 177, 'fancied': 112, 'up': 358, 'oktoberfest': 234, 'while': 371, 'good': 132, 


In [113]:
print(vectorizer.idf_)

[ 6.20350706  7.90825515  7.90825515 ...,  7.50279005  7.90825515
  7.90825515]


In [98]:
# encode document
vector = vectorizer.transform([X[0]])
# summarize encoded vector
print(vector.shape)
print(vector.toarray())

(1, 384)
[[ 0.01357063  0.01357063  0.0407119   0.01357063  0.02714126  0.01357063
   0.01357063  0.01357063  0.01357063  0.01357063  0.02714126  0.05428253
   0.13570632  0.01357063  0.01357063  0.01357063  0.08142379  0.01357063
   0.01357063  0.01357063  0.02714126  0.36640707  0.01357063  0.01357063
   0.02714126  0.01357063  0.01357063  0.0407119   0.01357063  0.01357063
   0.14927696  0.01357063  0.01357063  0.01357063  0.02714126  0.01357063
   0.01357063  0.01357063  0.01357063  0.05428253  0.13570632  0.01357063
   0.01357063  0.01357063  0.02714126  0.01357063  0.01357063  0.01357063
   0.01357063  0.01357063  0.02714126  0.0407119   0.01357063  0.01357063
   0.01357063  0.0407119   0.02714126  0.0407119   0.01357063  0.10856506
   0.02714126  0.02714126  0.02714126  0.01357063  0.01357063  0.01357063
   0.0407119   0.02714126  0.01357063  0.01357063  0.01357063  0.01357063
   0.02714126  0.06785316  0.0407119   0.01357063  0.0407119   0.02714126
   0.0407119   0.01357063  0.

In [104]:
# vectorize with a list of reviews

from sklearn.feature_extraction.text import TfidfVectorizer
X = df['review'][0:2000]

# create the transform
vectorizer = TfidfVectorizer()
# tokenize and build vocab
vectorizer.fit(X)
# summarize
print(str(vectorizer.vocabulary_)[:1000])  # print a slice because this is very long
print(len(str(vectorizer.vocabulary_)))

{'smell': 11165, 'soft': 11263, 'hop': 6090, 'aroma': 918, 'with': 13621, 'significant': 10951, 'malt': 7438, 'scents': 10579, 'this': 12373, 'one': 8499, 'smells': 11171, 'very': 13165, 'creamy': 3179, 'taste': 12183, 'and': 760, 'it': 6575, 'is': 6562, 'the': 12317, 'traditional': 12594, 'irish': 6546, 'flavors': 4964, 'come': 2790, 'out': 8606, 'at': 1005, 'tongue': 12522, 'not': 8299, 'like': 7146, 'cream': 3171, 'ale': 629, 'but': 2030, 'close': 2653, 'big': 1452, 'buttery': 2039, 'smooth': 11194, 'hops': 6116, 'are': 896, 'unique': 12939, 'sharp': 10812, 'flavor': 4957, 'an': 750, 'easy': 4133, 'saturated': 10524, 'well': 13435, 'mixed': 7865, 'blend': 1560, 'that': 12312, 'plays': 9177, 'complimenting': 2866, 'second': 10666, 'fiddle': 4818, 'to': 12482, 'base': 1222, 'no': 8248, 'sweetness': 12034, 'finish': 4878, 'nutty': 8357, 'changes': 2388, 'personalities': 9001, 'end': 4289, 'mouthfeel': 8004, 'lightly': 7135, 'carbonated': 2203, 'exceptionally': 4515, 'drinkability': 400

In [23]:
from sklearn.feature_extraction.text import HashingVectorizer

X = [df['review'][2]]  # start with a long review
X2 = [df['review'][0]] # X2 is shorter

# create the transform
vectorizer = HashingVectorizer(n_features=50)
# encode document
vector = vectorizer.transform(X)
# summarize encoded vector
print(vector.shape)
print(vector.toarray())

(1, 50)
[[ 0.07887912  0.01314652 -0.0657326   0.02629304 -0.0657326  -0.17090475
  -0.01314652  0.         -0.28922343  0.01314652  0.10517215  0.01314652
   0.28922343 -0.14461171 -0.01314652  0.01314652 -0.02629304 -0.10517215
   0.21034431  0.          0.07887912  0.14461171 -0.07887912  0.02629304
  -0.21034431  0.07887912 -0.14461171  0.14461171  0.01314652  0.
   0.02629304 -0.03943956  0.24978387  0.14461171  0.07887912  0.15775823
   0.07887912 -0.02629304  0.23663735  0.13146519 -0.22349083  0.19719779
   0.         -0.03943956  0.07887912 -0.36810254  0.36810254 -0.07887912
  -0.03943956  0.01314652]]


In [31]:
counts = df.groupby(['style']).size()
styles = counts.index

In [32]:
styles

Index(['Altbier', 'American Adjunct Lager', 'American Amber / Red Ale',
       'American Amber / Red Lager', 'American Barleywine',
       'American Black Ale', 'American Blonde Ale', 'American Brown Ale',
       'American Dark Wheat Ale', 'American Double / Imperial IPA',
       ...
       'Scotch Ale / Wee Heavy', 'Scottish Ale',
       'Scottish Gruit / Ancient Herbed Ale', 'Smoked Beer', 'Tripel',
       'Vienna Lager', 'Weizenbock', 'Wheatwine', 'Winter Warmer', 'Witbier'],
      dtype='object', name='style', length=104)

In [51]:
# vectorize with a list of reviews
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

#  Create X for predictor variables, y for target
X = df['review'][0:2000].values
y = df['style'][0:2000].values
test_X = df['review'][3000:4000].values
test_y = df['style'][3000:4000].values

# count_vect = CountVectorizer()
# X_train_counts = count_vect.fit_transform(X)

vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X)

# Naive Bayes classifier
clf = MultinomialNB().fit(X_train, y)

In [52]:
predicted = clf.predict(test_X)
np.mean(predicted == test_y)

ValueError: could not convert string to float: "   poured from a 650ml bottle,7% abv, 80 ibu's fresh hopped with cascade and centennial with nugget for bittering. l: pours a dark red amber with just a skim of head with minor lacing. s: not much of a nose for me ,some hops and mild spice. t: really smooth well balance red ipa, looks like they used 5 malts and it works well with the fresh hops nice on the bitterness. f: a light carbonation nice mouth feel no alcohol taste at all. o: a really nice fresh hopped red ipa.   "

In [ ]:
# could use a pipeline to do the same:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),                      
                     ('clf', MultinomialNB()),])
text_clf = text_clf.fit(X,y)

### start over here

In [60]:
df = df_copy

In [66]:
#prep the data
df = df_copy
df = df.drop(['name','brewery'], axis=1)
# drop all reviews with < 15 characters
df = df[df['review'].map(len) > 14]

In [68]:
print(len(df))

49291


In [69]:
t_id = 0
text = df.loc[t_id, 'review']

In [72]:
from gensim.summarization import summarize, keywords

word_scores = keywords(text, words=5, scores=True, split=True, lemmatize=True)

ModuleNotFoundError: No module named 'gensim'